## Deliverable 2. Create a Customer Travel Destinations Map.

In [50]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
# Import API key
from config import g_key
from config import weather_api_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [51]:
#1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()
  

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Yellowknife,CA,2021-10-10 15:16:50,62.4560,-114.3525,36.41,100,90,10.36
1,1,Imbituba,BR,2021-10-10 15:16:50,-28.2400,-48.6703,69.69,75,100,10.83
2,2,Punta Arenas,CL,2021-10-10 15:16:32,-53.1500,-70.9167,50.11,43,20,27.63
3,3,Lebu,CL,2021-10-10 15:16:51,-37.6167,-73.6500,54.27,74,95,1.36
4,4,Pilar,BR,2021-10-10 15:15:18,-9.5972,-35.9567,85.93,47,96,10.60


In [52]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Whats is the minimum temperature you would like for your trip?"))
max_temp = float(input("Whats is the maximum temperature you would like for your trip?"))

Whats is the minimum temperature you would like for your trip?70
Whats is the maximum temperature you would like for your trip?80


In [53]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
8,8,Caravelas,BR,2021-10-10 15:16:52,-17.7125,-39.2481,76.95,72,100,8.61
9,9,Hilo,US,2021-10-10 15:12:36,19.7297,-155.0900,75.27,92,90,3.44
12,12,Atuona,PF,2021-10-10 15:16:53,-9.8000,-139.0333,78.04,72,3,19.33
17,17,Rikitea,PF,2021-10-10 15:16:54,-23.1203,-134.9692,74.37,81,100,19.33
21,21,San Cristobal,VE,2021-10-10 15:16:55,7.7669,-72.2250,77.18,77,96,1.95
23,23,Oistins,BB,2021-10-10 15:16:55,13.0667,-59.5333,78.89,89,75,20.71
28,28,Kapaa,US,2021-10-10 15:16:57,22.0752,-159.3190,74.12,79,75,5.01
31,31,Haimen,CN,2021-10-10 15:16:57,23.1935,116.6142,78.69,83,9,5.88
44,44,Bambous Virieux,MU,2021-10-10 15:17:01,-20.3428,57.7575,73.65,88,75,16.11
45,45,Viedma,AR,2021-10-10 15:17:02,-40.8135,-62.9967,76.50,24,0,15.12


In [54]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       135
City          135
Country       135
Date          135
Lat           135
Lng           135
Max Temp      135
Humidity      135
Cloudiness    135
Wind Speed    135
dtype: int64

In [55]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID       135
City          135
Country       135
Date          135
Lat           135
Lng           135
Max Temp      135
Humidity      135
Cloudiness    135
Wind Speed    135
dtype: int64

In [56]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Caravelas,BR,76.95,-17.7125,-39.2481,
9,Hilo,US,75.27,19.7297,-155.0900,
12,Atuona,PF,78.04,-9.8000,-139.0333,
17,Rikitea,PF,74.37,-23.1203,-134.9692,
21,San Cristobal,VE,77.18,7.7669,-72.2250,
23,Oistins,BB,78.89,13.0667,-59.5333,
28,Kapaa,US,74.12,22.0752,-159.3190,
31,Haimen,CN,78.69,23.1935,116.6142,
44,Bambous Virieux,MU,73.65,-20.3428,57.7575,
45,Viedma,AR,76.50,-40.8135,-62.9967,


In [57]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
"radius": 5000,
"type": "lodging",
"key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
# 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  

# 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel is not found... skipping")

hotel_df.head()


Hotel is not found... skipping
Hotel is not found... skipping
Hotel is not found... skipping
Hotel is not found... skipping
Hotel is not found... skipping


,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Caravelas,BR,76.95,-17.7125,-39.2481,Pousada dos Navegantes
9,Hilo,US,75.27,19.7297,-155.0900,Hilo Hawaiian Hotel
12,Atuona,PF,78.04,-9.8000,-139.0333,Villa Enata
17,Rikitea,PF,74.37,-23.1203,-134.9692,People ThankYou
21,San Cristobal,VE,77.18,7.7669,-72.2250,Posada Villaven C.A.


In [58]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count

<bound method DataFrame.count of               City Country  Max Temp      Lat       Lng  \
8        Caravelas      BR     76.95 -17.7125  -39.2481   
9             Hilo      US     75.27  19.7297 -155.0900   
12          Atuona      PF     78.04  -9.8000 -139.0333   
17         Rikitea      PF     74.37 -23.1203 -134.9692   
21   San Cristobal      VE     77.18   7.7669  -72.2250   
..             ...     ...       ...      ...       ...   
545    Barra Velha      BR     71.60 -26.6322  -48.6847   
554      Kalangala      UG     79.95  -0.3089   32.2250   
555        Kimitsu      JP     74.08  35.3253  139.8911   
557      Machaneng      BW     77.52 -23.1862   27.4886   
559         Beloha      MG     71.56 -25.1667   45.0500   

                  Hotel Name  
8     Pousada dos Navegantes  
9        Hilo Hawaiian Hotel  
12               Villa Enata  
17           People ThankYou  
21      Posada Villaven C.A.  
..                       ...  
545  Vila Alaide Praia Hotel  
554     Ss

In [59]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [61]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [62]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))